In [1]:
import pandas as pd
import sqlite3
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

In [3]:
import pandas as pd
c.execute("""SELECT * FROM employees""")
df = pd.DataFrame(c.fetchall())
df.head()

,0,1,2,3,4,5,6,7
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


Selected customers from a certain city.

In [4]:
c.execute("""SELECT * FROM customers WHERE city = 'Boston';""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,362,Gifts4AllAges.com,Yoshido,Juri,6175559555,8616 Spinnaker Dr.,,Boston,MA,51003,USA,1216,41900.00
1,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100.00


Selected customers from multiple cities.

In [5]:
c.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",,Madrid,,28034,Spain,1370,227600.00
1,237,ANG Resellers,Camino,Alejandra,(91) 745 6555,"Gran Vía, 1",,Madrid,,28001,Spain,,0.00
2,344,CAF Imports,Fernandez,Jesus,+34 913 728 555,Merchants House,27-30 Merchant's Quay,Madrid,,28023,Spain,1702,59600.00
3,362,Gifts4AllAges.com,Yoshido,Juri,6175559555,8616 Spinnaker Dr.,,Boston,MA,51003,USA,1216,41900.00
4,458,"Corrida Auto Replicas, Ltd",Sommer,Martín,(91) 555 22 82,"C/ Araquil, 67",,Madrid,,28023,Spain,1702,104600.00
5,465,"Anton Designs, Ltd.",Anton,Carmen,+34 913 728555,"c/ Gobelas, 19-1 Urb. La Florida",,Madrid,,28023,Spain,,0.00
6,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100.00


Added a conditional for customer credit limit and limited the amount of results. 

In [6]:
c.execute("""SELECT *
             FROM customers
             WHERE (city = 'Boston' or city = 'Madrid')
                   AND (creditLimit >= 50000.00)
             LIMIT 15
             ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",,Madrid,,28034,Spain,1370,227600.00
1,237,ANG Resellers,Camino,Alejandra,(91) 745 6555,"Gran Vía, 1",,Madrid,,28001,Spain,,0.00
2,344,CAF Imports,Fernandez,Jesus,+34 913 728 555,Merchants House,27-30 Merchant's Quay,Madrid,,28023,Spain,1702,59600.00
3,362,Gifts4AllAges.com,Yoshido,Juri,6175559555,8616 Spinnaker Dr.,,Boston,MA,51003,USA,1216,41900.00
4,458,"Corrida Auto Replicas, Ltd",Sommer,Martín,(91) 555 22 82,"C/ Araquil, 67",,Madrid,,28023,Spain,1702,104600.00


Selected customers and ordered by Customer name. 

In [7]:
c.execute("""SELECT *
             FROM customers
             ORDER BY customerName
             ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,237,ANG Resellers,Camino,Alejandra,(91) 745 6555,"Gran Vía, 1",,Madrid,,28001,Spain,,0.00
1,187,"AV Stores, Co.",Ashworth,Rachel,(171) 555-1555,Fauntleroy Circus,,Manchester,,EC2 5NT,UK,1501,136800.00
2,242,Alpha Cognac,Roulet,Annette,61.77.6555,1 rue Alsace-Lorraine,,Toulouse,,31000,France,1370,61100.00
3,168,American Souvenirs Inc,Franco,Keith,2035557845,149 Spinnaker Dr.,Suite 101,New Haven,CT,97823,USA,1286,0.00
4,249,Amica Models & Co.,Accorti,Paolo,011-4988555,Via Monte Bianco 34,,Torino,,10100,Italy,1401,113000.00


Joining two tables on officecode column, isolated Bostom employees. 

In [8]:
c.execute("""SELECT * FROM employees
                        JOIN offices
                        ON employees.officeCode = offices.officeCode
                        WHERE city == 'Boston';
                       """)
df = pd.DataFrame(c.fetchall()) 
df.columns = [i[0] for i in c.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


In [10]:
c.execute("""SELECT * FROM products
                        LEFT JOIN orderdetails
                        USING(productCode);
                       """)
df = pd.DataFrame(c.fetchall()) 
df.columns = [i[0] for i in c.description]
print(len(df))
df.head()

2997


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,orderNumber,quantityOrdered,priceEach,orderLineNumber
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10107,30,81.35,2
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10121,34,86.13,5
2,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10134,41,90.92,2
3,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10145,45,76.56,6
4,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10159,49,81.35,14


Locating what city has the most employees per office, and put them in descending order. 

In [12]:
c.execute("""SELECT city,
                      COUNT(employeeNumber) AS numEmployees
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY city
                      ORDER BY numEmployees desc;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


In [13]:
c.execute("""SELECT * FROM products
                        LEFT JOIN orderdetails
                        USING(productCode);
                       """)
df = pd.DataFrame(c.fetchall()) 
df.columns = [i[0] for i in c.description]
print(len(df))
df.head()

2997


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,orderNumber,quantityOrdered,priceEach,orderLineNumber
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10107,30,81.35,2
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10121,34,86.13,5
2,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10134,41,90.92,2
3,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10145,45,76.56,6
4,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10159,49,81.35,14


Created an empty list to append employees whose stock ownership is within 500-10000 shares. 

In [15]:
l=[]
for i in df.quantityInStock:
   i=int(i)
   l.append(i)
df.quantityInStock=l
df.query('10000>quantityInStock >500').head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,orderNumber,quantityOrdered,priceEach,orderLineNumber
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10107,30,81.35,2
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10121,34,86.13,5
2,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10134,41,90.92,2
3,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10145,45,76.56,6
4,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10159,49,81.35,14
